In [2]:
import earthaccess
import xarray as xr
import time
from functools import wraps
!pip install --upgrade --force-reinstall h5coro==0.0.8

# Authenticate for accessing NASA data
auth = earthaccess.login(strategy=".netrc")

  Using cached h5coro-0.0.8-py3-none-any.whl.metadata (815 bytes)
  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached boto3-1.40.14-py3-none-any.whl.metadata (6.7 kB)
  Using cached earthaccess-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached multimethod-2.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached pqdm-0.2.0-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached python_cmr-0.13.0-py3-none-any.whl.metadata (10 kB)
  Using cached s3fs-2025.7.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached tinynetrc-1.3.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached charset_normalizer-3.4.3-cp311-cp311-manylinux2014_x86_64.manylinux_

In [3]:
# Confirm h5coro backend loaded properly
# should see 'h5coro' in the list
xr.backends.list_engines().keys()

dict_keys(['netcdf4', 'h5netcdf', 'scipy', 'h5coro', 'kerchunk', 'rasterio', 'store', 'zarr'])

In [4]:
s3url_atl06 = 'nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2019/12/02/ATL06_20191202203649_10220511_006_01.h5'
auth = earthaccess.login()
creds = auth.get_s3_credentials(daac='NSIDC')

In [5]:
%%timeit -r 5

ds = xr.open_dataset(s3url_atl06, engine='h5coro', group='/gt2l/land_ice_segments', credentials=creds)
ds.h_li.load()
ds.latitude.load()
ds.longitude.load()
ds.delta_time.load()

1.51 s ± 185 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [7]:
%%timeit -r 5

short_name="ATL06",
granule_name='ATL06_20191202203649_10220511_006_01.h5',
group='/gt2l/land_ice_segments'

results = earthaccess.search_data(
    short_name=short_name,
    granule_name=granule_name,
)

s3_urls = results[0].data_links(access="direct")
granule_s3_url = s3_urls[0][3:]
ds = xr.open_dataset(granule_s3_url, engine='h5coro', group='/gt2l/land_ice_segments', credentials=creds)
ds.h_li.load()
ds.latitude.load()
ds.longitude.load()
ds.delta_time.load()

2.75 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
%%timeit -r 1

short_name="ATL06",
granule_name='ATL06_20191202203649_10220511_006_01.h5',
group='/gt2l/land_ice_segments'

results = earthaccess.search_data(
    short_name=short_name,
    granule_name=granule_name,
)

opened = earthaccess.open(results)
ds = xr.open_dataset(opened[0], engine="h5netcdf", group='/gt2l/land_ice_segments')
ds.h_li.load()
ds.latitude.load()
ds.longitude.load()
ds.delta_time.load()

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

2min 2s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
